In [1]:
# run the cell first for all setup

import re
from re import search
import os, sys
from os import walk
import traceback
import subprocess
import pandas as pd
import time
import math
import numpy as np

from tqdm import tqdm

def minmaxavg(numlist):
    remove_list=[0,100]
    numlist=[i for i in numlist if i <100]
    numlist=[i for i in numlist if i >0]
    numlist.sort()
    try:
        lstavg=sum(numlist)/len(numlist)
    except:
        lstavg=np.nan
    try:
        lstavgT1=sum(numlist[-1:])/1
    except:
        lstavgT1=np.nan
    try:
        lstavgT3=sum(numlist[-3:])/3
    except:
        lstavgT3=np.nan
    try:
        lstavgT5=sum(numlist[-5:])/5
    except:
        lstavgT5=np.nan
    try:
        lstavgT7=sum(numlist[-7:])/7
    except:
        lstavgT7=np.nan
    try:
        lstmin=min(numlist)
    except:
        lstmin=np.nan
    try:
        lstmax=max(numlist)
    except:
        lstmax=np.nan
        
    return(lstmin,lstmax,lstavg,lstavgT1,lstavgT3,lstavgT5,lstavgT7)

def seq_similarity (seq1_list,seq2_list,select_opt=''):
    s=0
    t=0
    t_ALL=0
    s_ALL=0
    for i,aa1 in enumerate(seq1_list):
        aa1=int(seq1_list[i])
        aa2=int(seq2_list[i])
        
        t_ALL=t_ALL+1
        if aa1==aa2:
            s_ALL=s_ALL+1
            
        if aa1+aa2 != 0:
            t=t+1
            if aa1==aa2:
                s=s+1
                
    try:
        ss=(s/t)*100
    except:
        ss=0
        
    try:
        ss_ALL=(s_ALL/t_ALL)*100
    except:
        ss_ALL=0
        
    try:
        ss_NOZERO=(s/t_ALL)*100
    except:
        ss_NOZERO=0
        
    if select_opt=='ALLSS':
        print ('ss_ALLSS',s_ALL,t_ALL,ss_ALL)
        ss_out=ss_ALL
    elif select_opt=='NOZERO':
        print ('ss_NOZERO',s,t_ALL,ss_NOZERO)
        ss_out=ss_NOZERO
    else:
        print ('ss',s,t,ss)
        ss_out=ss
        
    return(ss_out)

def pair_seqCUT_similarty(workpath,ppalignfile,cutnum,seqsimCCnames,peotein_index,top):
    timestamp=time.strftime("%Y%m%d",time.localtime(time.time()))
    alignmentDF=pd.DataFrame()
    os.chdir(workpath)
    protalign=pd.read_csv(ppalignfile,header=None ) 
    protalign1=protalign.T.set_index(1).T
    prot_conv=protalign1.to_dict('list')
    protalign_output='similarty_proteinpairsimilarity'+timestamp+'_'+top+'_'+re.sub('.csv','',ppalignfile)+'_'+str(cutnum)+'CUT'+'.csv'
    alnum=0
    for i,ppp1 in enumerate (tqdm(protalign1.columns)):
        for j,ppp2 in enumerate (protalign1.columns):
            print ('\n --- ',ppp1,ppp2, ' --- ')
            seqsim_name_num=0
            sscut=[]
            seq1_list0=[]
            seq2_list0=[]
            
            seq1=prot_conv[ppp1][1]
            seq2=prot_conv[ppp2][1]
            
            ######### no CUT seq_similarity
            try:
                seq1_list0 = [(re.sub('\s|\[|\]|\'','',x)) for x in seq1.split(',')]
                seq2_list0 = [(re.sub('\s|\[|\]|\'','',x)) for x in seq2.split(',')]
            except:
                seq1_list0=[]
                seq2_list0=[]
                print ('not avalable seq1_list seq2_list',ppp1,ppp2)
                continue

            alignmentDF.at[alnum,'seq1']=ppp1
            alignmentDF.at[alnum,'seq2']=ppp2
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            select_opt=''
            seq_similarity_out=seq_similarity(seq1_list,seq2_list,select_opt=select_opt)
            sscut.append(seq_similarity_out)
            try:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
            except:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
            seqsim_name_num=seqsim_name_num+1
            
            ######### CUT seq_similarity
            
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            try:
                seqcutSTEP=int(len(seq1_list)/(cutnum))
                passtail=0
                for startlist in range (0,int(len(seq1_list)),seqcutSTEP):
                    if passtail==1:
                        continue
                    stoplist_next=startlist+seqcutSTEP+seqcutSTEP
                    if stoplist_next < len(seq1_list): 
                        stoplist=startlist+seqcutSTEP
                        seq1_list_cut=seq1_list[startlist:stoplist]
                        seq2_list_cut=seq2_list[startlist:stoplist]
                    else:
                        stoplist=len(seq1_list)
                        seq1_list_cut=seq1_list[startlist:]
                        seq2_list_cut=seq2_list[startlist:]
                        passtail=1

                    select_opt=''
                    seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
                    sscut.append(seq_similarity_out)
                    try:
                        alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                        print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
                    except:
                        alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                        print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
                    seqsim_name_num=seqsim_name_num+1
            except:
                seqcutSTEP=int(len(seq1_list))
                print (' *** ERROR protein sequence cannot be cutted *** seqcutSTEP: ',seqcutSTEP)
                
                
            ######### NOZERO no CUT seq_similarity
            try:
                seq1_list0 = [(re.sub('\s|\[|\]|\'','',x)) for x in seq1.split(',')]
                seq2_list0 = [(re.sub('\s|\[|\]|\'','',x)) for x in seq2.split(',')]
            except:
                seq1_list0=[]
                seq2_list0=[]
                print ('not avalable seq1_list seq2_list',ppp1,ppp2)
                continue

            alignmentDF.at[alnum,'seq1']=ppp1
            alignmentDF.at[alnum,'seq2']=ppp2
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            select_opt='NOZERO'
            seq_similarity_out=seq_similarity(seq1_list,seq2_list,select_opt=select_opt)
            sscut.append(seq_similarity_out)
            try:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
            except:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
            seqsim_name_num=seqsim_name_num+1
            
            ######### NOZERO CUT seq_similarity
            
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            try:
                seqcutSTEP=int(len(seq1_list)/(cutnum))
                passtail=0
                for startlist in range (0,int(len(seq1_list)),seqcutSTEP):
                    if passtail==1:
                        continue
                    stoplist_next=startlist+seqcutSTEP+seqcutSTEP
                    if stoplist_next < len(seq1_list): 
                        stoplist=startlist+seqcutSTEP
                        seq1_list_cut=seq1_list[startlist:stoplist]
                        seq2_list_cut=seq2_list[startlist:stoplist]
                    else:
                        stoplist=len(seq1_list)
                        seq1_list_cut=seq1_list[startlist:]
                        seq2_list_cut=seq2_list[startlist:]
                        passtail=1

                    select_opt='NOZERO'
                    seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
                    sscut.append(seq_similarity_out)
                    try:
                        alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                        print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
                    except:
                        alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                        print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
                    seqsim_name_num=seqsim_name_num+1
            except:
                seqcutSTEP=int(len(seq1_list))
                print (' *** ERROR protein sequence cannot be cutted *** seqcutSTEP: ',seqcutSTEP)
                

            #########  TOP feature  no CUT seq_similarity
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            seq1_list_cut=[]
            seq2_list_cut=[]
            seq1_list_copy=seq1_list.copy()
            seq2_list_copy=seq2_list.copy()
            
            for pp in peotein_index:
                seq1_list_cut.append(seq1_list_copy.pop(pp))
                seq2_list_cut.append(seq2_list_copy.pop(pp))
            
            select_opt=''
            seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
            sscut.append(seq_similarity_out)
            try:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
            except:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
            seqsim_name_num=seqsim_name_num+1
            
            ######### TOP feature select CUT seq_similarity
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            passtail=0
            for startlist in range (0,int(len(seq1_list)),seqcutSTEP):
                seq1_list_cut=[]
                seq2_list_cut=[]
                seq1_list_copy=seq1_list.copy()
                seq2_list_copy=seq2_list.copy()
                
                if passtail==1:
                    continue
                stoplist_next=startlist+seqcutSTEP+seqcutSTEP
                if stoplist_next < len(seq1_list): 
                    stoplist=startlist+seqcutSTEP
                    for pp in peotein_index:
                        if (pp >= startlist) and (pp < stoplist):
                            seq1_list_cut.append(seq1_list_copy.pop(pp))
                            seq2_list_cut.append(seq2_list_copy.pop(pp))
                else:
                    stoplist=len(seq1_list)
                    for pp in peotein_index:
                        if (pp >= startlist) and (pp < stoplist):
                            seq1_list_cut.append(seq1_list_copy.pop(pp))
                            seq2_list_cut.append(seq2_list_copy.pop(pp))
                    passtail=1

                select_opt=''
                seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
                sscut.append(seq_similarity_out)
                try:
                    alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                    print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
                except:
                    alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                    print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
                seqsim_name_num=seqsim_name_num+1
            
            #########
            
            
            ####### ALLSS ##  TOP feature  no CUT seq_similarity
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            seq1_list_cut=[]
            seq2_list_cut=[]
            seq1_list_copy=seq1_list.copy()
            seq2_list_copy=seq2_list.copy()

            for pp in peotein_index:
                seq1_list_cut.append(seq1_list_copy.pop(pp))
                seq2_list_cut.append(seq2_list_copy.pop(pp))
                
            select_opt='ALLSS'
            seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
            sscut.append(seq_similarity_out)
            try:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
            except:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
            seqsim_name_num=seqsim_name_num+1
            
            
            ####### ALLSS ## TOP feature select CUT seq_similarity
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            passtail=0
            for startlist in range (0,int(len(seq1_list)),seqcutSTEP):
                seq1_list_cut=[]
                seq2_list_cut=[]
                seq1_list_copy=seq1_list.copy()
                seq2_list_copy=seq2_list.copy()
                
                if passtail==1:
                    continue
                stoplist_next=startlist+seqcutSTEP+seqcutSTEP
                if stoplist_next < len(seq1_list): 
                    stoplist=startlist+seqcutSTEP
                    for pp in peotein_index:
                        if (pp >= startlist) and (pp < stoplist):
                            seq1_list_cut.append(seq1_list_copy.pop(pp))
                            seq2_list_cut.append(seq2_list_copy.pop(pp))
                else:
                    stoplist=len(seq1_list)
                    for pp in peotein_index:
                        if (pp >= startlist) and (pp < stoplist):
                            seq1_list_cut.append(seq1_list_copy.pop(pp))
                            seq2_list_cut.append(seq2_list_copy.pop(pp))
                    passtail=1

                select_opt='ALLSS'
                seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
                sscut.append(seq_similarity_out)
                try:
                    alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                    print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
                except:
                    alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                    print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
                seqsim_name_num=seqsim_name_num+1

                
                
                
            ###### NOZERO ##  TOP feature  no CUT seq_similarity
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            seq1_list_cut=[]
            seq2_list_cut=[]
            seq1_list_copy=seq1_list.copy()
            seq2_list_copy=seq2_list.copy()
            for pp in peotein_index:
                seq1_list_cut.append(seq1_list_copy.pop(pp))
                seq2_list_cut.append(seq2_list_copy.pop(pp))
        
            select_opt='NOZERO'
            seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
            sscut.append(seq_similarity_out)
            try:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
            except:
                alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
            seqsim_name_num=seqsim_name_num+1
            
            
            ####### NOZERO ## TOP feature select CUT seq_similarity
            seq1_list=seq1_list0.copy()
            seq2_list=seq2_list0.copy()
            
            passtail=0
            for startlist in range (0,int(len(seq1_list)),seqcutSTEP):
                seq1_list_cut=[]
                seq2_list_cut=[]
                seq1_list_copy=seq1_list.copy()
                seq2_list_copy=seq2_list.copy()
                
                if passtail==1:
                    continue
                stoplist_next=startlist+seqcutSTEP+seqcutSTEP
                if stoplist_next < len(seq1_list): 
                    stoplist=startlist+seqcutSTEP
                    for pp in peotein_index:
                        if (pp >= startlist) and (pp < stoplist):
                            seq1_list_cut.append(seq1_list_copy.pop(pp))
                            seq2_list_cut.append(seq2_list_copy.pop(pp))
                else:
                    stoplist=len(seq1_list)
                    for pp in peotein_index:
                        if (pp >= startlist) and (pp < stoplist):
                            seq1_list_cut.append(seq1_list_copy.pop(pp))
                            seq2_list_cut.append(seq2_list_copy.pop(pp))
                    passtail=1

                select_opt='NOZERO'
                seq_similarity_out=seq_similarity(seq1_list_cut,seq2_list_cut,select_opt=select_opt)
                sscut.append(seq_similarity_out)
                try:
                    alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]="{:.3f}".format(seq_similarity_out)
                    print (seqsimCCnames[seqsim_name_num],seq_similarity_out,select_opt)
                except:
                    alignmentDF.at[alnum,seqsimCCnames[seqsim_name_num]]=np.nan
                    print (seqsimCCnames[seqsim_name_num],' check nan',select_opt)
                seqsim_name_num=seqsim_name_num+1

            alnum=alnum+1

        os.chdir(workpath)
        print ('OUTPUT to ',protalign_output,ppp1)
        alignmentDF.to_csv(protalign_output,index=False,encoding='utf-8')

    os.chdir(workpath)
    print ('DONE ALL_OUTPUT to ',protalign_output,workpath)
    alignmentDF.to_csv(protalign_output,index=False,encoding='utf-8')
    return(protalign_output)

def proteinname_seqCUT_similarty(workpath,protalign_output,cutnum,seqsimCCnames,peotein_index,top):
    os.chdir(workpath)
   
    alignmentDF_Interspecies=pd.read_csv(protalign_output)
    
    workpath_featureselect='/home1/drhuangwc/AAALAB/GPCR5b/Interspecies_20220915_AgonistAntagonist_datacode2ALLcolumns/GPCR_to_test_FeatureSelection0616'
    trainingdata=[f for f in next(walk(workpath_featureselect))[2] if '_valid.csv' in f]

    alignmentDF_VALID=pd.DataFrame()
    print (trainingdata)
    for includedata in trainingdata:
        includeprotein=re.sub('GPCR_to_trainAgonistAntagonist_','',includedata)
        includeprotein=re.sub('_valid.csv','',includeprotein)
        alignmentDF_LOC=alignmentDF_Interspecies.loc[alignmentDF_Interspecies['seq2']==includeprotein]
        if len(alignmentDF_LOC)==0:
            print ('No data: ',includeprotein)
        alignmentDF_VALID=pd.concat([alignmentDF_VALID,alignmentDF_LOC],ignore_index=True)
        alignmentDF_VALID=alignmentDF_VALID.reset_index(drop=True)
    
    alignmentDF=alignmentDF_VALID.copy()
    alignmentDF_group=alignmentDF.groupby(['seq1'])
    similartydf=pd.DataFrame()
    seqsimCCnames=[]
    for cc in alignmentDF.columns:
        if re.search('seq_similarity',cc):
            seqsimCCnames.append(cc)
    proteinsimilarity_output=re.sub('.csv','',protalign_output)+top+'_SIMminmaxavg.csv'
    ppnum=0
    for i,ppp in enumerate(tqdm(alignmentDF_group.first().index)):
        ppp_groupDF=alignmentDF_group.get_group(ppp)
        similartydf.at[ppnum,'protein']=ppp
        lstmin_list=[]
        lstmax_list=[]
        lstavg_list=[]
        lstavgT5_list=[]
        for cut,seq_simnum in enumerate(seqsimCCnames):
            seq_similarity_name=re.sub('seq_similarity','',seq_simnum)
            similarityslst=ppp_groupDF[seq_simnum].to_list()
            lstmin,lstmax,lstavg,lstavgT1,lstavgT3,lstavgT5,lstavgT7=minmaxavg(numlist=similarityslst)

            lstavgT1_name='SIMavgT1_'+seq_similarity_name 
            lstavgT3_name='SIMavgT3_'+seq_similarity_name 
            lstavgT5_name='SIMavgT5_'+seq_similarity_name
            lstavgT7_name='SIMavgT7_'+seq_similarity_name 
            lstmin_name='SIMmin_'+seq_similarity_name 
            lstavg_name='SIMavg_'+seq_similarity_name 
            lstmax_name='SIMmax_'+seq_similarity_name 

            try:
                similartydf.at[ppnum,lstavgT1_name]=lstavgT1
            except:
                similartydf.at[ppnum,lstavgT1_name]=np.nan
            try:
                similartydf.at[ppnum,lstavgT3_name]=lstavgT3
            except:
                similartydf.at[ppnum,lstavgT3_name]=np.nan
            try:
                similartydf.at[ppnum,lstavgT5_name]=lstavgT5
            except:
                similartydf.at[ppnum,lstavgT5_name]=np.nan
            try:
                similartydf.at[ppnum,lstavgT7_name]=lstavgT7
            except:
                similartydf.at[ppnum,lstavgT7_name]=np.nan
            try:
                similartydf.at[ppnum,lstmax_name]=lstmax
            except:
                similartydf.at[ppnum,lstmax_name]=np.nan
            try:
                similartydf.at[ppnum,lstmin_name]=lstmin
            except:
                similartydf.at[ppnum,lstmin_name]=np.nan
            try:
                similartydf.at[ppnum,lstavg_name]=lstavg
            except:
                similartydf.at[ppnum,lstavg_name]=np.nan
                
        ppnum=ppnum+1
    os.chdir(workpath)
    similartydf.to_csv(proteinsimilarity_output,index=False,encoding='utf-8')
    return(proteinsimilarity_output)

homepath=str(os.getcwd())




### CODE: Main


In [ ]:

######################################## Main ########################################

cutnum=7

timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()


workpath=homepath+'Human_AgonistAntagonist_dataprocess/GPCR_similarty'

os.chdir(workpath)

ppalignfile='proteinseq20220915_align.csv'

MLfile_train='GPCR_to_trainAgonistAntagonist_humanalltrainconcat_selefeatureTOP200.csv'
top='TOP200'

protalign_output='similarty_proteinpairsimilarity20230624_TOP200_proteinseq20220915_align_7CUT.csv'

datatrain=pd.read_csv(MLfile_train)
peotein_index_TOP200=[]
for p in datatrain.columns:
    if re.search('Protein_code1',p):
        pp=re.sub('Protein_code1_','',p)
        peotein_index_TOP200.append(int(pp))
peotein_index_TOP200.sort(reverse=True)
print ('selefeatureTOP200 : ',peotein_index_TOP200)

i=0
print ('cut=',cutnum)
seqsimCCnames_TOP200=[]
if cutnum<2:
    cutnum=1
    cutnum_range=1
else:
    cutnum_range=cutnum+1
for i in range (cutnum_range):
    listname='seq_similarityCUT'+str(cutnum)+'_'+str(i)
    seqsimCCnames_TOP200.append(listname)

for i in range (cutnum_range):
    listname='seq_similarityNOZEROCUT'+str(cutnum)+'_'+str(i)
    seqsimCCnames_TOP200.append(listname)
    
for i in range (cutnum_range):
    listname='seq_similarityTOP200CUT'+str(cutnum)+'_'+str(i)
    seqsimCCnames_TOP200.append(listname)

for i in range (cutnum_range):
    listname='seq_similarityTOP200ALLSSCUT'+str(cutnum)+'_'+str(i)
    seqsimCCnames_TOP200.append(listname)
    
for i in range (cutnum_range):
    listname='seq_similarityTOP200NOZEROCUT'+str(cutnum)+'_'+str(i)
    seqsimCCnames_TOP200.append(listname)
    
print (seqsimCCnames_TOP200)


os.chdir(workpath)

proteinsimilarity_output=proteinname_seqCUT_similarty(workpath,protalign_output,cutnum,seqsimCCnames_TOP200,peotein_index_TOP200,top=top)


print ('done TOP200',cutnum,'\n output to : \n',protalign_output,' \n',proteinsimilarity_output)


if (time.time()-starttime)/3600 > 1 :
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
else:
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )


##### result data:

In [2]:
workpath=homepath+'/Human_AgonistAntagonist_dataprocess/GPCR_similarty'
os.chdir(workpath)
similartydf=pd.read_csv('similarty_proteinpairsimilarity20230624_TOP200_proteinseq20220915_align_7CUTTOP200_SIMminmaxavg.csv')
similartydf

,protein,SIMavgT1_CUT7_0,SIMavgT3_CUT7_0,SIMavgT5_CUT7_0,SIMavgT7_CUT7_0,SIMmax_CUT7_0,SIMmin_CUT7_0,SIMavg_CUT7_0,SIMavgT1_CUT7_1,SIMavgT3_CUT7_1,...,SIMmax_TOP200NOZEROCUT7_6,SIMmin_TOP200NOZEROCUT7_6,SIMavg_TOP200NOZEROCUT7_6,SIMavgT1_TOP200NOZEROCUT7_7,SIMavgT3_TOP200NOZEROCUT7_7,SIMavgT5_TOP200NOZEROCUT7_7,SIMavgT7_TOP200NOZEROCUT7_7,SIMmax_TOP200NOZEROCUT7_7,SIMmin_TOP200NOZEROCUT7_7,SIMavg_TOP200NOZEROCUT7_7
0,5HT1A_HUMAN,53.273,53.273000,53.1904,53.096000,53.273,11.438,33.082649,50.000,46.154000,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,5HT1B_HUMAN,66.259,66.259000,64.8978,63.342143,66.259,12.171,35.010802,40.000,39.762000,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,5HT1D_HUMAN,66.259,66.259000,64.5808,62.662857,66.259,11.281,34.607683,47.619,46.031667,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,5HT1E_HUMAN,59.453,59.453000,59.1360,58.773714,59.453,10.393,34.520874,38.095,38.095000,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,5HT1F_HUMAN,64.829,64.829000,63.0190,60.950429,64.829,10.729,34.507159,53.333,51.241667,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,UR2R_HUMAN,45.894,45.563000,45.1904,44.799143,45.894,12.581,35.019738,40.541,40.541000,...,NaN,NaN,NaN,50.0,50.0,50.0,50.0,50.0,50.0,50.0
212,V1AR_HUMAN,55.862,50.202000,46.7592,45.042000,55.862,12.000,33.167247,40.741,40.741000,...,NaN,NaN,NaN,50.0,50.0,50.0,50.0,50.0,50.0,50.0
213,V1BR_HUMAN,52.902,49.088000,45.9842,44.458714,52.902,11.653,32.928901,47.059,44.257667,...,NaN,NaN,NaN,50.0,50.0,50.0,50.0,50.0,50.0,50.0
214,V2R_HUMAN,53.317,47.907000,46.1188,44.796429,53.317,10.339,33.878194,46.429,46.337333,...,NaN,NaN,NaN,50.0,50.0,50.0,50.0,50.0,50.0,50.0
